In [1]:
# Importing necessary libraries
import mlflow
import mlflow.keras
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import Callback
import shap
import numpy as np
import matplotlib.pyplot as plt
import os


### Configuration and Data Loading

In this section, we load the MNIST dataset and perform normalization.


In [2]:
# Loading the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalizing data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Expanding dimensions to add a grayscale channel
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

input_shape = x_train.shape[1:]


### Model Construction

We build a convolutional neural network model to classify MNIST images.


In [3]:
# Building the deep learning model
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

learning_rate = 0.1
# Setting up the optimizer
optimizer = optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

c:\Users\EnricPereraiBel\T2C\AI_CHAPTER\git\mlflow_uni_demo\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Model Training

We define the training parameters and train the model, using MLflow to log metrics.


In [4]:
# Training parameters
epochs = 5
batch_size = 64

# Starting an MLflow experiment
mlflow.start_run()

# Logging parameters to MLflow
mlflow.log_param("epochs", epochs)
mlflow.log_param("batch_size", batch_size)
mlflow.log_param("optimizer", "adam")
mlflow.log_param("learning_rate", learning_rate)

class MLflowEpochCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            mlflow.log_metric("train_accuracy", logs.get("accuracy"))
            mlflow.log_metric("train_loss", logs.get("loss"))
            mlflow.log_metric("val_accuracy", logs.get("val_accuracy"))
            mlflow.log_metric("val_loss", logs.get("val_loss"))
            mlflow.log_metric("epoch", epoch)
mlflow_callback = MLflowEpochCallback()
            
# Trainning the model
history = model.fit(x_train, y_train,
                    validation_data=(x_test, y_test),
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[mlflow_callback])

# Saving the model to MLflow
mlflow.keras.log_model(model, artifact_path="model")


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - accuracy: 0.1014 - loss: 4.8090 - val_accuracy: 0.0958 - val_loss: 2.3054
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.1065 - loss: 2.3101 - val_accuracy: 0.0980 - val_loss: 2.3072
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.1055 - loss: 2.3095 - val_accuracy: 0.1135 - val_loss: 2.3114
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.1013 - loss: 2.3096 - val_accuracy: 0.1135 - val_loss: 2.3115
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.1070 - loss: 2.3104 - val_accuracy: 0.1028 - val_loss: 2.3066


2024/11/15 18:46:47 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/11/15 18:46:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Model Evaluation

We evaluate the model on the test set and log the final metrics to MLflow.


In [5]:
# Evaluating the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_acc}')

# Logging final metrics to MLflow
mlflow.log_metric("test_loss", test_loss)
mlflow.log_metric("test_accuracy", test_acc)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1027 - loss: 2.3051
Test loss: 2.3065989017486572, Test accuracy: 0.10279999673366547


### Model Explanation with SHAP

We generate explanations for the model's predictions using SHAP and save them in MLflow.


In [6]:
# Creating a directory for SHAP images if it doesn't exist
shap_images_dir = "shap_images"
os.makedirs(shap_images_dir, exist_ok=True)

# Using SHAP to explain the model
background = x_train[np.random.choice(x_train.shape[0], 100, replace=False)]
test_images = x_test[:10]

# Creating the SHAP explainer and get SHAP values
explainer = shap.GradientExplainer(model, background)
shap_values = explainer.shap_values(test_images)

# Getting the model predictions
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Saving SHAP explanations in MLflow
for i in range(len(test_images)):
    shap_image = os.path.join(shap_images_dir, f"shap_explanation_{i}.png")
    shap_value = shap_values[i][:, :, :, predicted_classes[i]]
    
    # Visualizing and save the SHAP explanation
    shap.image_plot([shap_value], -test_images[i], show=False)
    plt.savefig(shap_image, bbox_inches='tight')
    plt.close()
    
    # Logging the image to MLflow
    mlflow.log_artifact(shap_image, artifact_path="shap_images")


c:\Users\EnricPereraiBel\T2C\AI_CHAPTER\git\mlflow_uni_demo\.venv\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: keras_tensor. Received: the structure of inputs=['*']
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


### End the MLflow Experiment

We end the experiment run in MLflow.


In [7]:
# Ending the MLflow run
mlflow.end_run()